In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import copy
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import category_encoders as ce

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support as score

import seaborn as sns
le = preprocessing.LabelEncoder()
scale = StandardScaler()

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Scale allContinuous numeric columns
# Use Standard Scaler to scale numerics for the model
def scalefeatures(df):
    for col in numCols: 
        colName = col.replace(' ', '') + '_scale' # Remove Spaces in Column names
        df[colName] = scale.fit_transform(df[[col]]) # Create new column with standard scaled value
        df = df.set_index(colName).reset_index() # Move column to the front of the DF
    df = df.set_index('object_id').reset_index()
    df = df.drop(numCols, axis=1)
    return df

# Sets the distmod to 0 and creates a new category for galactic (is milky way)
def distmodclean(df):
    # No records use 0 for distmod so will make them 0
    #find nan values to replace
    nanDistMod = np.isnan(df['distmod'])
    # print(nanDistMod)
    df['galactic'] = np.isnan(df['distmod'])
    df.loc[np.isnan(df['distmod']), 'distmod'] = 0

    # If Distmod has 0 create Milkyway resident
    df['galactic'] = le.fit_transform(df['galactic'])
    temp = df['galactic'] 
    df.drop(labels=['galactic'], axis=1,inplace = True)
    df.insert(len(df.columns.tolist())-2, 'galactic', temp)

In [ ]:
DATA_DIR = '../input/'
# DATA_DIR = '' # For Azure Notebook

trainMdDf = pd.read_csv(DATA_DIR+'training_set_metadata.csv')
testdf = pd.read_csv(DATA_DIR+'test_set_metadata.csv')

# hostgal_specz is mostly missing form the Test data so just drop
trainMdDf = trainMdDf.drop('hostgal_specz', axis=1)
testdf = testdf.drop('hostgal_specz', axis=1)